In [1]:
import torch
import torchvision.models as models
import torch.nn as nn


In [2]:
resnet_model = models.resnet50(pretrained=True)
resnet_model = nn.Sequential( *list(resnet_model.children())[:-1] )


/home/erzurumlu.1/miniconda3/envs/whereami/lib/python3.9/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/erzurumlu.1/miniconda3/envs/whereami/lib/python3.9/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [3]:
# Generate a random input tensor
input_tensor = torch.randn(5, 3, 224, 224)

# Pass the input tensor through the resnet_model
output_tensor = resnet_model(input_tensor)

# Print the output shape
print(output_tensor.shape)


torch.Size([5, 2048, 1, 1])


In [4]:
import torch
import torch.nn as nn

class ImageProjection(nn.Module):
    
    def __init__(self, image_embedding_size, shared_embedding_size, dropout=0.1):
        super(ImageProjection, self).__init__()
        self.image_projection = nn.Linear(image_embedding_size, shared_embedding_size)
        self.gelu = nn.GELU()
        self.fc = nn.Linear(shared_embedding_size, shared_embedding_size)
        self.dropout = nn.Dropout(dropout)
        self.layer_norm = nn.LayerNorm(shared_embedding_size)
        
    def forward(self, image_embeddings):
        projected_embeddings = self.image_projection(image_embeddings)
        
        x = self.gelu(projected_embeddings)
        x = self.fc(x)
        x = self.dropout(x)
        x = x + projected_embeddings
        x = self.layer_norm(x)
        
        return x

In [5]:
image_projection = ImageProjection(2048, 512)
output_tensor = image_projection(output_tensor.squeeze())
print(output_tensor.shape)

torch.Size([5, 512])


In [2]:
from equiconv_encoder import PanoramaVisionEncoder

layerdict, offsetdict = torch.load('/home/erzurumlu.1/yunus/git/whereami/checkpoints/panorama_unet/offset_layer/layer_256x512-b128.pt'), torch.load('/home/erzurumlu.1/yunus/git/whereami/checkpoints/panorama_unet/offset_layer/offset_256x512-b128.pt')
model = PanoramaVisionEncoder(layer_dict=layerdict, offset_dict=offsetdict)
input_tensor = torch.randn(128, 3, 256, 512)
model.load_state_dict(torch.load('/home/erzurumlu.1/yunus/git/whereami/checkpoints/panorama_unet/equi_panorama_encoder.pth'))
output_tensor = model(input_tensor)
print(output_tensor.shape)

torch.Size([128, 2048, 1, 1])
